In [15]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from skimage import measure, morphology
import os #per poter prendere le immagini nella cartella di input e iterare 

# Percorsi
input_folder = 'FESI/Progetto/malaria/malaria/'  # Cartella delle immagini di input
output_folder = 'FESI/Progetto/malaria/output_Metodo1/'  # Cartella dove salvare le immagini elaborate

# Crea le cartelle di output se non esistono
os.makedirs(output_folder, exist_ok=True)

# Itera sui file nella cartella di input
for filename in os.listdir(input_folder):
    if filename.endswith('.png') or filename.endswith('.jpg'):  # Filtra i tipi di file immagine
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)
        resized = cv2.resize(img, (224, 224)) #ridimensionamento dell'immagine
        img_gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY) # Conversione in scala di grigi

        # Creo una maschera per rilevare il contenuto non nero
        _, mask = cv2.threshold(img_gray, 10, 255, cv2.THRESH_BINARY)

        resized_rgba = cv2.cvtColor(resized, cv2.COLOR_BGR2RGBA)
        result = resized_rgba.copy()
        result[mask == 0] = [255,255,255,0]   # Seleziono tutti i pixel che corrispondono a dove mask è 0 (quindi dove mask è nera) e li metto bianchi

        #in questa immagine c'è un quarto valore dopo 255 perchè sto usando RGBA --> A è il canale alfa per la trasparenza

        #Durante la conversione in scala di grigi il canale alfa (che gestisce la trasparenza) viene perso, di conseguenza lo sfondo torna al suo colore originario (io ho impostato bianco)
        #Posso evitare che ciò succeda conservando il canale alfa, e riapplicandolo all'immagine convertita. 
        #Tuttavia la funzione cv2.equalizeHist accetta solo immagini a un singolo canale (grayscale), di conseguenza non posso passare come argomento  un'immagine con due canali (grigio + alfa), che non perchè supportata da equalizeHist.
        # Quindi effettuo prima l'equalizzazione unicamente sul canale grigio, e in seguito applico anche il canale alfa separatamente 

        # Separo i quattro canali RGBA
        r, g, b, alpha = cv2.split(result)

        #converto l'immagine in scala di grigi + canale alfa
        resultGray = cv2.cvtColor(cv2.merge((r, g, b)), cv2.COLOR_BGR2GRAY)

        # Equalizzazione dell'istogramma sul canale grigio
        eq_img = cv2.equalizeHist(resultGray)

        # Combino il tutto con il canale alfa
        bgra_gray = cv2.merge((eq_img, eq_img, eq_img, alpha))

        # Siccome non posso applicare la sogliatura sull'immagine RGBA, lavoro su un'immagine in scala di grigi e successivamente riapplico il canale alfa 
        img_noRGBA = cv2.cvtColor(bgra_gray, cv2.COLOR_BGR2GRAY)

        #Seconda sogliatura
        _, thresh = cv2.threshold(img_noRGBA, 8, 255, cv2.THRESH_BINARY)

        # Rimozione del rumore
        kernel = np.ones((3,3), np.uint8) #creo il kernel di convoluzione --> matrice 3x3
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,kernel, iterations = 2)

        # Identificazione e Labeling delle regioni con lo scopo di rimuovere le regioni troppo piccole
        labels = measure.label(opening, connectivity=2)  
        properties = measure.regionprops(labels)
        min_size = 700  # Cambia in base alle dimensioni delle macchie
        filtered_labels = morphology.remove_small_objects(labels, min_size=min_size)
        segmented = filtered_labels > 0  # Converti in maschera binaria

        mask = (cv2.bitwise_not(thresh)) > 0

        imgSegmented = resized.copy() #prendo una copia dell'originale sulla quale applicherò la maschera
        imgSegmented[mask] = [255, 255, 255] 

         # Salva la maschera e l'immagine modificata
        cv2.imwrite(output_path, imgSegmented)

        # (Opzionale) Visualizza il progresso
        print(f"Elaborata immagine: {filename}")




Elaborata immagine: parasite (1).png
Elaborata immagine: parasite (10).png
Elaborata immagine: parasite (100).png
Elaborata immagine: parasite (101).png
Elaborata immagine: parasite (102).png
Elaborata immagine: parasite (103).png
Elaborata immagine: parasite (104).png
Elaborata immagine: parasite (105).png
Elaborata immagine: parasite (106).png
Elaborata immagine: parasite (107).png
Elaborata immagine: parasite (108).png
Elaborata immagine: parasite (109).png
Elaborata immagine: parasite (11).png
Elaborata immagine: parasite (110).png
Elaborata immagine: parasite (111).png
Elaborata immagine: parasite (112).png


/tmp/ipykernel_6721/811542915.py:65: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  filtered_labels = morphology.remove_small_objects(labels, min_size=min_size)


Elaborata immagine: parasite (113).png
Elaborata immagine: parasite (114).png
Elaborata immagine: parasite (115).png
Elaborata immagine: parasite (116).png
Elaborata immagine: parasite (117).png
Elaborata immagine: parasite (118).png
Elaborata immagine: parasite (119).png
Elaborata immagine: parasite (12).png
Elaborata immagine: parasite (120).png
Elaborata immagine: parasite (121).png
Elaborata immagine: parasite (122).png
Elaborata immagine: parasite (123).png
Elaborata immagine: parasite (124).png
Elaborata immagine: parasite (125).png
Elaborata immagine: parasite (126).png
Elaborata immagine: parasite (127).png
Elaborata immagine: parasite (128).png
Elaborata immagine: parasite (129).png
Elaborata immagine: parasite (13).png
Elaborata immagine: parasite (130).png
Elaborata immagine: parasite (131).png
Elaborata immagine: parasite (132).png
Elaborata immagine: parasite (133).png
Elaborata immagine: parasite (134).png
Elaborata immagine: parasite (135).png
Elaborata immagine: parasit